In [ ]:
import sys
sys.path.append('../')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from datetime import date

In [ ]:
sys.path.append('C:\\Users\\olegk\\Documents\\unifloc\\')

In [ ]:
import uniflocpy.uTools.plotly_workflow as pw
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, plot

In [ ]:
test = pd.read_csv("edited_kobzar2.csv", index_col = "2")
test.index = pd.to_datetime(test.index)

In [ ]:
test_resampled = test.resample('3h').mean()

In [ ]:
test_resampled = test_resampled.dropna(subset = [' Объемный дебит жидкости'])

In [ ]:
all_traces = pw.create_traces_list_for_all_columms(test_resampled, chosen_mode = 'lines')

In [ ]:
data = all_traces
pw.plot_subplots(data, 'Исходные данные по 3 часа без интерполяции' + '.html', True)


In [ ]:
def plot_corr_heatmap(df, filename_str):
    corrs = df.corr()
    figure = ff.create_annotated_heatmap(
       z=corrs.values,
       x=list(corrs.columns),
       y=list(corrs.index),
       annotation_text=corrs.round(2).values,
       showscale=True)
    figure.layout.update(
        margin=go.layout.Margin(
            l=400,
            r=50,
            b=100,
            t=250
        ))
    filename_str +='.html'
    plot(figure,filename=filename_str)

In [ ]:
plot_corr_heatmap(test_resampled, "Корреляционная карта для данных по 3 часа без интерполяции")

In [ ]:
from os import walk
import os
current_path = os.getcwd()
path_to_data = current_path + "\\data\\"
dirnames_list = []
for (dirpath, dirnames, filenames) in walk(path_to_data):
    dirnames_list.extend(dirnames)
    break

In [ ]:
dirnames_list

In [ ]:
def initial_editing(data, wellname):
    if len(data.columns)==4:  
        del data[0]

        test_str = data[1][0]
        index = test_str.find(wellname)
        str_to_delete = test_str[:index] + wellname + '. '
        data[1] = data[1].str.replace(str_to_delete,"")
        data.index = data[2]
        del data[2]
    else:
        test_str = data[0][0]
        index = test_str.find(wellname)
        str_to_delete = test_str[:index] + wellname + '. '
        data[0] = data[0].str.replace(str_to_delete,"")
        data.index = data[1]
        del data[1]
    data.index = pd.to_datetime(data.index)
    return data

def extract_df_one_parametr_and_edit(data, list_of_params, number_of_param_in_list, patameters_column):
    extracted_df_one_param = data[data[patameters_column] == list_of_params[number_of_param_in_list]].copy()
    edited_df_one_param = extracted_df_one_param.rename(index=str, columns = {3: extracted_df_one_param[patameters_column][patameters_column]})
    del edited_df_one_param[patameters_column]
    return edited_df_one_param

def create_edited_df(data, parametrs_list):
    init_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, 0, patameters_column)
    result = init_one_parametr_df
    for i in range(1, len(parametrs_list)):
        new_one_parametr_df = extract_df_one_parametr_and_edit(data, parametrs_list, i, patameters_column)
        result = result.join([new_one_parametr_df], how = "outer", sort=True)
    return result

In [ ]:
files = ['1479', "1509"]
test_file = None

In [ ]:
for this_file_name in files:
    print("Первичная обработка данных со СУ для скважины " + this_file_name)
    
    this_file_path = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + ".csv"
    test_file = pd.read_csv(this_file_path,sep=';', header=None)
    if len(test_file.columns)==4:
        patameters_column = 1
    else:
        patameters_column = 0
    edited_file = initial_editing(test_file, this_file_name)
    big_list_parametrs = np.unique(edited_file[patameters_column])
    edited_file = create_edited_df(edited_file, big_list_parametrs)
    this_file_path_out = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + "_first_edit" +  ".csv"
    edited_file.to_excel(this_file_path_out)

In [ ]:



known_file = '1354'
known_file_path = current_path + "\\data\\" + known_file +  "\\" + known_file + ".csv"
known_file = pd.read_csv(known_file_path,sep=';', header=None)

In [ ]:
known_file.head()

In [ ]:
known_file.columns

In [ ]:
edited_file = initial_editing(test_file, "1354")

In [ ]:
big_list_parametrs = np.unique(edited_file[1])

In [ ]:

edited_file = create_edited_df(edited_file, big_list_parametrs)

In [ ]:
result_by_func.to_excel("check2.xlsx")

In [ ]:
file_name_problem = '1479'

In [ ]:
this_file_path = current_path + "\\data\\" + file_name_problem +  "\\" + file_name_problem + ".csv"
test_file = pd.read_csv(this_file_path,sep=';', header=None)

In [ ]:
test_file.columns

In [ ]:
test_file.head()

In [ ]:
edited_file = initial_editing(test_file, file_name_problem)

In [ ]:
this_file_name = '1479'

In [ ]:
this_file_path = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + ".csv"
test_file = pd.read_csv(this_file_path,sep=';', header=None)
if len(test_file.columns)==4:
    patameters_column = 1
else:
    patameters_column = 0
edited_file = initial_editing(test_file, this_file_name)
big_list_parametrs = np.unique(edited_file[patameters_column])
edited_file = create_edited_df(edited_file, big_list_parametrs)


In [ ]:
this_file_path_out = current_path + "\\data\\" + this_file_name +  "\\" + this_file_name + "_first_edit" +  ".csv"

In [ ]:
this_file_path_out

In [ ]:

edited_file.to_csv(this_file_path_out)